# AMMICO Demonstration Notebook
With ammico, you can analyze text on images and image content at the same time. This is a demonstration notebook to showcase the capabilities of ammico.
You can run this notebook on google colab or locally / on your own HPC resource. The first cell only runs on google colab; on all other machines, you need to create a conda environment first and install ammico from the Python Package Index using  
```pip install ammico```  
Alternatively you can install the development version from the GitHub repository  
```pip install git+https://github.com/ssciwr/AMMICO.git```

In [1]:
# if running on google colab
# flake8-noqa-cell
import os

if "google.colab" in str(get_ipython()):
    # update python version
    # install setuptools
    # %pip install setuptools==61 -qqq
    # install ammico
    %pip install git+https://github.com/ssciwr/ammico.git -qqq
    # mount google drive for data and API key
    from google.colab import drive

    drive.mount("/content/drive")

Import the ammico package.

In [2]:
import ammico

# Step 1: Read your data into AMMICO
The ammico package reads in one or several input files given in a folder for processing. The user can select to read in all image files in a folder, to include subfolders via the `recursive` option, and can select the file extension that should be considered (for example, only "jpg" files, or both "jpg" and "png" files). For reading in the files, the ammico function `find_files` is used, with optional keywords:

| input key | input type | possible input values |
| --------- | ---------- | --------------------- |
`path` | `str` | the directory containing the image files (defaults to the location set by environment variable `AMMICO_DATA_HOME`) |
| `pattern` | `str\|list` | the file extensions to consider (defaults to "png", "jpg", "jpeg", "gif", "webp", "avif", "tiff") |
| `recursive` | `bool` | include subdirectories recursively (defaults to `True`) |
| `limit` | `int` | maximum number of files to read (defaults to `20`, for all images set to `None` or `-1`) |
| `random_seed` | `str` | the random seed for shuffling the images; applies when only a few images are read and the selection should be preserved (defaults to `None`) |

The `find_files` function returns a nested dict that contains the file ids and the paths to the files and is empty otherwise. This dict is filled step by step with more data as each detector class is run on the data (see below).

In [3]:
image_dict = ammico.find_files(
    path="data/",
    limit=10,
)

## Step 2: Inspect the input files using the graphical user interface
A Dash user interface is to select the most suitable options for the analysis, before running a complete analysis on the whole data set. The options for each detector module are explained below in the corresponding sections; for example, different models can be selected that will provide slightly different results. This way, the user can interactively explore which settings provide the most accurate results. In the interface, the nested `image_dict` is passed through the `AnalysisExplorer` class. The interface is run on a specific port which is passed using the `port` keyword; if a port is already in use, it will return an error message, in which case the user should select a different port number. 
The interface opens a dash app inside the Jupyter Notebook and allows selection of the input file in the top left dropdown menu, as well as selection of the detector type in the top right, with options for each detector type as explained below. The output of the detector is shown directly on the right next to the image. This way, the user can directly inspect how updating the options for each detector changes the computed results, and find the best settings for a production run.

Please note that for the Google Cloud Vision API (the TextDetector class) you need to set a key in order to process the images. This key is ideally set as an environment variable using for example
```
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"
] = "/content/drive/MyDrive/misinformation-data/misinformation-campaign-981aa55a3b13.json"
```
where you place the key on your Google Drive if running on colab, or place it in a local folder on your machine.

In [4]:
analysis_explorer = ammico.AnalysisExplorer(image_dict)
analysis_explorer.run_server(port=8055)

## Step 3: Analyze all images
After having selected the best options for each detector module from the interactive GUI, the analysis can now be run in production on all images in the data set. Depending on the size of the data set and the computing resources available, this can take some time. Please note that you need to have set your Google Cloud Vision API key for the TextDetector to run.
The desired detector modules are called sequentially in any order, for example:

In [5]:
for key in image_dict.keys():
    image_dict[key] = ammico.TextDetector(image_dict[key], analyse_text=True).analyse_image()
    image_dict[key] = ammico.EmotionDetector(image_dict[key]).analyse_image()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/42.8 MB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 26.0/42.8 MB 138.8 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 63.7 MB/s eta 0:00:00


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


...

1/1 [==============================] - 0s 213ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 337ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 65ms/step


For the computationally demanding `SummaryDetector`, it is best to initialize the model first and then analyze each image while passing the model explicitly. This can be done in a separate loop or in the same loop as for text and emotion detection.

In [6]:
# initialize the models
summary_model, summary_vis_processors = ammico.SummaryDetector(image_dict).load_model(model_type="base")
# run the analysis without having to re-iniatialize the model
for key in image_dict.keys():
    image_dict[key] = ammico.SummaryDetector(image_dict[key], analysis_type="summary", 
                                             summary_model=summary_model, 
                                             summary_vis_processors=summary_vis_processors).analyse_image()

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 73.0MB/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 42.6kB/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

 95%|█████████▍| 1.28G/1.35G [00:07<00:00, 213MB/s]

 96%|█████████▌| 1.30G/1.35G [00:07<00:00, 215MB/s]

 98%|█████████▊| 1.32G/1.35G [00:07<00:00, 216MB/s]

100%|█████████▉| 1.34G/1.35G [00:07<00:00, 233MB/s]

100%|██████████| 1.35G/1.35G [00:07<00:00, 193MB/s]

This can be done in a separate loop or in the same loop as for text and emotion detection.

The nested dictionary will be updated from containing only the file id's and paths to the image files, to containing also all the image data.

## Step 4: Convert analysis output to pandas dataframe and write csv
The content of the nested dictionary can then conveniently be converted into a pandas dataframe for further analysis in Python, or be written as a csv file:

In [7]:
image_df = ammico.get_dataframe(image_dict)

Inspect the dataframe:

In [8]:
image_df.head(3)

,filename,text,text_language,text_english,text_clean,text_summary,sentiment,sentiment_score,entity,entity_type,...,multiple_faces,no_faces,wears_mask,age,gender,race,emotion,emotion (category),const_image_summary,3_non-deterministic_summary
0,data/106349S_por.png,NEWS URGENTE SAMSUNG AO VIVO Rio de Janeiro NO...,pt,NEWS URGENT SAMSUNG LIVE Rio de Janeiro NEW CO...,NEWS URGENT SAMSUNG LIVE Rio de Janeiro NEW CO...,"NEW COUNTING METHOD RJ City HALL EXCLUDES 1,1...",NEGATIVE,0.99,"[Rio de Janeiro, C, ##IT, ##Y, PLANALTO]","[LOC, ORG, LOC, ORG, LOC]",...,No,1,[Yes],[24],[Man],[None],[None],[None],a man wearing a face mask while looking at a c...,"[an image of man using a cellphone with mask, ..."
1,data/102141_2_eng.png,CORONAVIRUS QUARANTINE CORONAVIRUS OUTBREAK BE...,en,CORONAVIRUS QUARANTINE CORONAVIRUS OUTBREAK BE...,CORONAVIRUS QUARANTINE CORONAVIRUS OUTBREAK BE...,Coronavirus QUARANTINE CORONAVIRUS OUTBREAK,NEGATIVE,0.97,"[CORONAVIRUS, ##ARANTI, CORONAVIR, Co, ##ronavi]","[ORG, MISC, ORG, PER, ORG]",...,No,1,[Yes],[25],[Man],[None],[None],[None],"a collage of images including a corona sign, a...","[a collage of images including a fence, man an..."
2,data/102730_eng.png,400 DEATHS GET E-BOOK X AN Corporation ncy Ser...,en,400 DEATHS GET E-BOOK X AN Corporation ncy Ser...,DEATHS GET E - BOOK X AN Corporation Services ...,A municipal worker sprays disinfectant on his...,NEGATIVE,0.99,"[AN Corporation ncy Services, Ahmedabad, RE, #...","[ORG, LOC, PER, ORG]",...,No,1,[No],[27],[Man],[asian],[sad],[Negative],two people in blue coats spray disinfection a van,[a man and a woman in blue coats spraying wate...


Or write to a csv file:

In [9]:
image_df.to_csv("data_out.csv")

# The detector modules
The different detector modules with their options are explained in more detail in this section.
## Text detector
Text on the images can be extracted using the `TextDetector` class (`text` module). The text is initally extracted using the Google Cloud Vision API and then translated into English with googletrans. The translated text is cleaned of whitespace, linebreaks, and numbers using Python syntax and spaCy. 

<img src="../_static/text_detector.png" width="800" />

The user can set if the text should be further summarized, and analyzed for sentiment and named entity recognition, by setting the keyword `analyse_text` to `True` (the default is `False`). If set, the transformers pipeline is used for each of these tasks, with the default models as of 03/2023. Other models can be selected by setting the optional keyword `model_names` to a list of selected models, on for each task: `model_names=["sshleifer/distilbart-cnn-12-6", "distilbert-base-uncased-finetuned-sst-2-english", "dbmdz/bert-large-cased-finetuned-conll03-english"]` for summary, sentiment, and ner. To be even more specific, revision numbers can also be selected by specifying the optional keyword `revision_numbers` to a list of revision numbers for each model, for example `revision_numbers=["a4f8f3e", "af0f99b", "f2482bf"]`. 

Please note that for the Google Cloud Vision API (the TextDetector class) you need to set a key in order to process the images. This key is ideally set as an environment variable using for example:

`os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "misinformation-campaign-981aa55a3b13.json"`

where you place the key on your Google Drive if running on colab, or place it in a local folder on your machine.

Summarizing, the text detection is carried out using the following method call and keywords, where `analyse_text`, `model_names`, and `revision_numbers` are optional:


In [10]:
for key in image_dict.keys():
    image_dict[key] = ammico.TextDetector(image_dict[key], 
    analyse_text=True, model_names=["sshleifer/distilbart-cnn-12-6", 
    "distilbert-base-uncased-finetuned-sst-2-english", 
    "dbmdz/bert-large-cased-finetuned-conll03-english"], 
    revision_numbers=["a4f8f3e", "af0f99b", "f2482bf"]).analyse_image()

The models can be adapted interactively in the notebook interface and the best models can then be used in a subsequent analysis of the whole data set.

A detailed description of the output keys and data types is given in the following table.

| output key | output type | output value |
| ---------- | ----------- | ------------ |
| `text` | `str` | the extracted text in the original language |
| `text_language` | `str` | the detected dominant language of the extracted text |
| `text_english` | `str` | the text translated into English |
| `text_clean` | `str` | the text after cleaning from numbers and unrecognizable words |
| `text_summary` | `str` | the summary of the text, generated with a transformers model |
| `sentiment` | `str` | the detected sentiment, generated with a transformers model |
| `sentiment_score` | `float` | the confidence associated with the predicted sentiment |
| `entity` | `list[str]` | the detected named entities, generated with a transformers model |
| `entity_type` | `list[str]` | the detected entity type |

## Image summary and query

The `SummaryDetector` can be used to generate image captions (`summary`) as well as visual question answering (`VQA`). 

<img src="../_static/summary_detector.png" width="800" />

This module is based on the [LAVIS](https://github.com/salesforce/LAVIS) library. Since the models can be quite large, an initial object is created which will load the necessary models into RAM/VRAM and then use them in the analysis. The user can specify the type of analysis to be performed using the `analysis_type` keyword. Setting it to `summary` will generate a caption (summary), `questions` will prepare answers (VQA) to a list of questions as set by the user, `summary_and_questions` will do both. Note that the desired analysis type needs to be set here in the initialization of the 
detector object, and not when running the analysis for each image; the same holds true for the selected model.

In [11]:
image_summary_detector = ammico.SummaryDetector(image_dict, analysis_type="summary", model_type="base")

The implemented models are listed below.

| input model name | model |
| ---------------- | ----- |
| base | BLIP image captioning base, ViT-B/16, pretrained on COCO dataset |
| large | BLIP image captioning large, ViT-L/16, pretrained on COCO dataset |
| vqa | BLIP base model fine-tuned on VQA v2.0 dataset |
| blip2_t5_pretrain_flant5xxl | BLIP2 pretrained on FlanT5<sub>XXL</sub> | 
| blip2_t5_pretrain_flant5xl | BLIP2 pretrained on FlanT5<sub>XL</sub> | 
| blip2_t5_caption_coco_flant5xl | BLIP2 pretrained on FlanT5<sub>XL</sub>, fine-tuned on COCO | 
| blip2_opt_pretrain_opt2.7b | BLIP2 pretrained on OPT-2.7b |
| blip2_opt_pretrain_opt6.7b | BLIP2 pretrained on OPT-6.7b | 
| blip2_opt_caption_coco_opt2.7b | BLIP2 pretrained on OPT-2.7b, fine-tuned on COCO | 
| blip2_opt_caption_coco_opt6.7b | BLIP2 pretrained on OPT-6.7b, fine-tuned on COCO |

For VQA, a list of questions needs to be passed when carrying out the analysis; these should be given as a list of strings.

In [12]:
list_of_questions = [
    "How many persons on the picture?",
    "Are there any politicians in the picture?",
    "Does the picture show something from medicine?",
]

Summarizing, the detector is run as

In [13]:
image_summary_vqa_detector = ammico.SummaryDetector(image_dict, analysis_type="summary_and_questions", 
                                                    model_type="base")
for key in image_dict.keys():
    image_dict[key] = image_summary_vqa_detector.analyse_image(image_dict[key], 
                                                               list_of_questions = list_of_questions)

The output is given as a dictionary with the following keys and data types:

| output key | output type | output value |
| ---------- | ----------- | ------------ |
| `const_image_summary` | `str` | when `analysis_type="summary"` or `"summary_and_questions"`, constant image caption (does not change upon re-running the analysis for the same model) |
| `3_non-deterministic_summary` | `list[str]` | when `analysis_type="summary"` or s`ummary_and_questions`, three different captions generated with different random seeds |
| *a user-defined input question* | `str` | when `analysis_type="questions"` or `summary_and_questions`, the answer to the user-defined input question | 

## Detection of faces and facial expression analysis
Faces and facial expressions are detected and analyzed using the `EmotionDetector` class from the `faces` module. Initially, it is detected if faces are present on the image using RetinaFace, followed by analysis if face masks are worn (Face-Mask-Detection). The detection of age, gender, race, and emotions is carried out with deepface.

<img src="../_static/emotion_detector.png" width="800" />

Depending on the features found on the image, the face detection module returns a different analysis content: If no faces are found on the image, all further steps are skipped and the result `"face": "No", "multiple_faces": "No", "no_faces": 0, "wears_mask": ["No"], "age": [None], "gender": [None], "race": [None], "emotion": [None], "emotion (category)": [None]` is returned. If one or several faces are found, up to three faces are analyzed if they are partially concealed by a face mask. If yes, only age and gender are detected; if no, also race, emotion, and dominant emotion are detected. In case of the latter, the output could look like this: `"face": "Yes", "multiple_faces": "Yes", "no_faces": 2, "wears_mask": ["No", "No"], "age": [27, 28], "gender": ["Man", "Man"], "race": ["asian", None], "emotion": ["angry", "neutral"], "emotion (category)": ["Negative", "Neutral"]`, where for the two faces that are detected (given by `no_faces`), some of the values are returned as a list with the first item for the first (largest) face and the second item for the second (smaller) face (for example, `"emotion"` returns a list `["angry", "neutral"]` signifying the first face expressing anger, and the second face having a neutral expression).

The emotion detection reports the seven facial expressions angry, fear, neutral, sad, disgust, happy and surprise. These emotions are assigned based on the returned confidence of the model (between 0 and 1), with a high confidence signifying a high likelihood of the detected emotion being correct. Emotion recognition is not an easy task, even for a human; therefore, we have added a keyword `emotion_threshold` signifying the % value above which an emotion is counted as being detected. The default is set to 50%, so that a confidence above 0.5 results in an emotion being assigned. If the confidence is lower, no emotion is assigned. 

From the seven facial expressions, an overall dominating emotion category is identified: negative, positive, or neutral emotion. These are defined with the facial expressions angry, disgust, fear and sad for the negative category, happy for the positive category, and surprise and neutral for the neutral category.

A similar threshold as for the emotion recognition is set for the race detection, `race_threshold`, with the default set to 50% so that a confidence for the race above 0.5 only will return a value in the analysis. 

Summarizing, the face detection is carried out using the following method call and keywords, where `emotion_threshold` and 
`race_threshold` are optional:

In [14]:
for key in image_dict.keys():
    image_dict[key] = ammico.EmotionDetector(image_dict[key], emotion_threshold=50, race_threshold=50).analyse_image()

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 1s 502ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 234ms/step


1/1 [==============================] - ETA: 0s

...

1/1 [==============================] - 0s 221ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 217ms/step


1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 14ms/step


The thresholds can be adapted interactively in the notebook interface and the optimal value can then be used in a subsequent analysis of the whole data set.

The output keys that are generated are

| output key | output type | output value |
| ---------- | ----------- | ------------ |
| `face` | `str` | if a face is detected |
| `multiple_faces` | `str` | if multiple faces are detected |
| `no_faces` | `int` | the number of detected faces |
| `wears_mask` | `list[str]` | if each of the detected faces wears a face covering, up to three faces |
| `age` | `list[int]` | the detected age, up to three faces |
| `gender` | `list[str]` | the detected gender, up to three faces |
| `race` | `list[str]` | the detected race, up to three faces, if above the confidence threshold |
| `emotion` | `list[str]` | the detected emotion, up to three faces, if above the confidence threshold |
| `emotion (category)` | `list[str]` | the detected emotion category (positive, negative, or neutral), up to three faces, if above the confidence threshold |

## Further detector modules
Further detector modules exist, such as `ColorDetector` and `MultimodalSearch`, also it is possible to carry out a topic analysis on the text data, as well as crop social media posts automatically. These are more experimental features and have their own demonstration notebooks.